<a href="https://colab.research.google.com/github/Fatikha-Imamagzam/Fingerprint_analisys/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Oct 20 10:52:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import joblib # to load KNNMmdels
from glob import glob
import cv2
import numpy as np
from keras.models import load_model 
import tensorflow as tf 


end = ["\033[0;30;1m","\033[1;31;1m","\033[1;32;1m", "\033[1;33;1m", "\033[1;34;1m", "\033[1;35;1m"] 

In [ ]:
def getRawDataSet(dataset_cat_path):   
    print(f"{end[1]}Data Alınıyor....") 
    cat = []
    for image in dataset_cat_path:
        img  = cv2.imread(image)
        img = cv2.resize(img, (32, 32), interpolation = cv2.INTER_AREA)
        cat.append(img) 
    print(f"{end[2]}Hazırlandı!{end[0]}")
    print(f"Toplam Resimler: {len(cat)}  {end[0]}\n\n")
    return cat

def testTrainedModels(data, classes, models, models_path):    
    print(f"{end[2]}Gender (Confidence) \t {end[3]}Age (Confidence) \t{end[4]}Race (Confidence){end[0]}")
    if len(data.shape)>1:
        gender_model = joblib.load(models_path+models[0]) 
        gender_predictions = gender_model.predict(data)

        age_model = joblib.load(models_path+models[1]) 
        age_predictions = age_model.predict(data)

        race_model = joblib.load(models_path+models[2]) 
        race_predictions = race_model.predict(data)
        
        pred_prob_gender = gender_model.predict_proba(data) 
        pred_prob_age  = age_model.predict_proba(data)
        pred_prob_race= race_model.predict_proba(data)

        #print(pred)
        itr = 0
        for i,j,k in zip(gender_predictions,age_predictions,race_predictions):              
            gender = classes[0][i]
            gender_prob = max(pred_prob_gender[itr])*100
            age = classes[1][j]            
            age_prob = max(pred_prob_age[itr])*100
            race = classes[2][k] 
            race_prob = max(pred_prob_race[itr])*100

            print(f"{end[2]}{gender[2]} ({gender_prob} %) \t {end[3]}{age} ({age_prob} %) \t{end[4]}{race} ({race_prob} %){end[0]}")
            
            #print(f"{end[1]}pred: {i}  |{end[2]}  Gender: {gender} {gender_prob}%  | {end[3]} Age: {age}  {age_prob}%  |  {end[4]}  Race: {race}  {race_prob}%") 
            itr += 1
            
    elif len(data.shape)==1:
        data = data.reshape(1,-1)
        gender_model = joblib.load(models_path+models[0]) 
        gender_predictions = gender_model.predict(data)
        print(gender_predictions)

        age_model = joblib.load(models_path+models[1]) 
        age_predictions = age_model.predict(data)

        race_model = joblib.load(models_path+models[2]) 
        race_predictions = race_model.predict(data)

        #print(pred)
        for i,j,k in zip(gender_predictions,age_predictions,race_predictions):  
            gender = classes[0][i]
            age = classes[1][j]
            race = classes[2][k]
            
            print(f"{gender} {age} {race}")        
    else:
        print(f"{end[1]}Invalid data!")
    
    return [gender, age, race]


def getPredictions(custom_model,image_path, class_names, image_size, pred_amount):
    
    img = tf.keras.utils.load_img(
        image_path, target_size=(image_size,image_size)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = custom_model.predict(img_array,verbose = 0)
    score = tf.nn.softmax(predictions[0])
    #print(predictions,"\n", score)

    """print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )"""
    return  [predictions, np.argmax(score), class_names[np.argmax(score)], "{:.2f}".format(100 * np.max(score))]



**Data alma**

In [ ]:
%%time
img_path =   glob("/content/drive/MyDrive/Test/*") 

CPU times: user 1.25 ms, sys: 71 µs, total: 1.32 ms
Wall time: 2.02 ms


#Alexnet



**Önceden eğitilmiş model alma**

In [ ]:
model_name= '/content/drive/MyDrive/SON_models_Finger/model_AlexNet_gender_500_weights_upSampling.best.hdf5'
loaded_model_gender = load_model(model_name)
loaded_model_gender 

model_name= '/content/drive/MyDrive/SON_models_Finger/model_AlexNet_age_500_weights_upSampling.best.hdf5'
loaded_model_age = load_model(model_name)
loaded_model_age
model_name= '/content/drive/MyDrive/SON_models_Finger/model_AlexNet_race_500_weights_upSampling.best.hdf5'
loaded_model_race = load_model(model_name)
loaded_model_race

In [ ]:
classes = [["female", "male"],["0_30", "31_40", "41_50", "51_above"], ["white", "asian", "african-american"]]

**Gercek zamanda Test**

In [ ]:
%%time
image_size = 227
pred_amount = 5
counter = 0 
print(f"{end[2]}Gender (Confidence) \t {end[3]}Age (Confidence) \t{end[4]}Race (Confidence){end[0]}")
for image in img_path:
    pred_gender = getPredictions(loaded_model_gender,image, classes[0], image_size, pred_amount)      
    pred_age = getPredictions(loaded_model_age, image, classes[1], image_size, pred_amount) 
    pred_race = getPredictions(loaded_model_race, image, classes[2], image_size, pred_amount) 


    print(f"{end[2]}{pred_gender[2]} ({pred_gender[3]} %) \t {end[3]}{pred_age[2]} ({pred_age[3]} %) \t{end[4]}{pred_race[2]} ({pred_race[3]} %){end[0]}")


    counter +=1 
    if counter==pred_amount:
        break
    

print("\n\n")

Gender (Confidence) 	 Age (Confidence) 	Race (Confidence)
female (73.11 %) 	 51_above (47.54 %) 	african-american (57.61 %)



CPU times: user 156 ms, sys: 10.3 ms, total: 167 ms
Wall time: 489 ms


# KNN

**Önceden eğitilmiş model alma**

In [ ]:
%%time
images = getRawDataSet(img_path) 

Data Alınıyor....
Hazırlandı!
Toplam Resimler: 1  


CPU times: user 6.45 ms, sys: 1.5 ms, total: 7.95 ms
Wall time: 10.3 ms


In [ ]:
data = np.array(images,dtype=float)
dataset_size = data.shape[0]
data = data.reshape(dataset_size,-1)


models = ['KNNdownSgenderModel.pkl','KNNdownSageModel.pkl','KNNdownSraceModel.pkl']
          #['KNNupSageModel.pkl', 'KNNupSgenderModel.pkl', 'KNNupSraceModel.pkl']
          #[KNNaugageModel.pkl', 'KNNauggenderModel.pkl', 'KNNaugraceModel.pkl']

print(f"{end[2]}Gender (Confidence) \t {end[3]}Age (Confidence) \t{end[4]}Race (Confidence){end[0]}")
predictions = testTrainedModels(data[0:2], classes, models, img_path)


Gender (Confidence) 	 Age (Confidence) 	Race (Confidence)
Gender (Confidence) 	 Age (Confidence) 	Race (Confidence)


TypeError: ignored

In [ ]:
data = np.array(male_images,dtype=float)
dataset_size = data.shape[0]
data = data.reshape(dataset_size,-1)


models = ['KNNdownSgenderModel.pkl','KNNdownSageModel.pkl','KNNdownSraceModel.pkl']
          #['KNNupSageModel.pkl', 'KNNupSgenderModel.pkl', 'KNNupSraceModel.pkl']
          #[KNNaugageModel.pkl', 'KNNauggenderModel.pkl', 'KNNaugraceModel.pkl']

testTrainedModels(data[9:12], classes, models, models_path)

pred: 0  |  Gender: female 100.0%  |  Age: 51_above  60.0%  |    Race: white  60.0%
pred: 0  |  Gender: female 80.0%  |  Age: 31_40  60.0%  |    Race: asian  60.0%
pred: 0  |  Gender: female 100.0%  |  Age: 0_30  40.0%  |    Race: white  60.0%


'pred: 0 Gender: female   Age: 0_30    Race: white'

In [ ]:
break here

SyntaxError: invalid syntax (760050616.py, line 1)